Importing libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

In [2]:
from sklearn.model_selection import train_test_split                              # to split the data into two parts
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics                                                       # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

In [3]:
## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Preprocessing

In [5]:
# importing dataset
df = pd.read_csv('/content/drive/MyDrive/SML_project/household_power_consumption.txt', sep=";", parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='dt')
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [6]:
df = df.fillna(method='ffill')
df.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [7]:
df['energy_consumed'] = (df['Global_active_power'] * 1000 / 60) - (df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3'])

In [9]:
data_resampled = df.resample('SM').sum()
data_resampled.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,energy_consumed
dt,,,,,,,,
2006-12-15,38332.010,2739.412,4965281.53,161961.8,27536.0,48403.0,156485.0,406442.833333
2006-12-31,34634.820,3090.402,5195859.08,146674.4,19277.0,36935.0,144067.0,376968.000000
2007-01-15,35512.854,2797.424,5558097.23,150307.0,37156.0,36736.0,176994.0,340994.900000
2007-01-31,34341.684,2543.752,5194277.02,144951.6,29936.0,40371.0,172991.0,329063.400000
2007-02-15,23996.272,2110.592,4500232.82,101306.6,17648.0,29843.0,110396.0,242050.866667


Converting time series into supervised machine learning problem

In [8]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	dff = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(dff.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(dff.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

#### Feature Scaling:

In [10]:
#Minmax standard scaling technique:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data_resampled.values)

# frame as supervised learning
data_reframed = series_to_supervised(scaled, 1, 1)

In [11]:
data_reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),var8(t)
1,0.950534,0.352012,0.554706,0.963389,0.626331,0.939737,0.586722,1.000000,0.847420,0.462026,0.705259,0.859667,0.438472,0.680222,0.531019,0.919942
2,0.847420,0.462026,0.705259,0.859667,0.438472,0.680222,0.531019,0.919942,0.871909,0.370195,0.941778,0.884313,0.845146,0.675718,0.678720,0.822234
3,0.871909,0.370195,0.941778,0.884313,0.845146,0.675718,0.678720,0.822234,0.839245,0.290684,0.704226,0.847978,0.680921,0.757977,0.660763,0.789826
4,0.839245,0.290684,0.704226,0.847978,0.680921,0.757977,0.660763,0.789826,0.550714,0.154914,0.251057,0.551854,0.401419,0.519733,0.379980,0.553488
5,0.550714,0.154914,0.251057,0.551854,0.401419,0.519733,0.379980,0.553488,0.567277,0.228134,0.707229,0.570262,0.607997,0.940032,0.388135,0.500320


In [18]:
# drop columns we don't want to predict
data_reframed.drop(data_reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(data_reframed.head())

IndexError: ignored

### Splitting dataset to train and test:

In [13]:
data_reframed.shape

(94, 9)

In [19]:
def split():
  values = data_reframed.values

  num_trains = 94-18
  #train data
  train = values[:num_trains, :]
  #test data
  test = values[num_trains:, :]

  # split into train and test
  train_X, train_y = train[:, :-1], train[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
  test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

  return train_X, train_y, test_X, test_y

In [22]:
models = {}
for unit in [40,70,100] :  
  train_X, train_y, test_X, test_y = split()
  model = Sequential()
  model.add(LSTM(unit, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')

  # fit network
  history = model.fit(train_X, train_y, epochs=80, batch_size=20, validation_data=(test_X, test_y), verbose=2, shuffle=False)

  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], 8))

  # invert scaling for forecast
  inv_yhat = np.concatenate((yhat, test_X[:, -7:]), axis=1)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]

  # invert scaling for actual
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = np.concatenate((test_y, test_X[:, -7:]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  
  # calculate RMSE
  rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
  mse = mean_squared_error(inv_y, inv_yhat)
  mae = mean_absolute_error(inv_y, inv_yhat)
  r2 = r2_score(inv_y, inv_yhat)
  mape = mean_absolute_percentage_error(inv_y, inv_yhat)

  models[unit] = [rmse, mse, mae, r2, mape]

Epoch 1/80
4/4 - 3s - loss: 0.2864 - val_loss: 0.1739 - 3s/epoch - 838ms/step
Epoch 2/80
4/4 - 0s - loss: 0.2245 - val_loss: 0.1342 - 35ms/epoch - 9ms/step
Epoch 3/80
4/4 - 0s - loss: 0.1813 - val_loss: 0.1008 - 32ms/epoch - 8ms/step
Epoch 4/80
4/4 - 0s - loss: 0.1347 - val_loss: 0.0736 - 29ms/epoch - 7ms/step
Epoch 5/80
4/4 - 0s - loss: 0.1049 - val_loss: 0.0524 - 36ms/epoch - 9ms/step
Epoch 6/80
4/4 - 0s - loss: 0.0715 - val_loss: 0.0369 - 35ms/epoch - 9ms/step
Epoch 7/80
4/4 - 0s - loss: 0.0507 - val_loss: 0.0266 - 35ms/epoch - 9ms/step
Epoch 8/80
4/4 - 0s - loss: 0.0465 - val_loss: 0.0208 - 59ms/epoch - 15ms/step
Epoch 9/80
4/4 - 0s - loss: 0.0290 - val_loss: 0.0185 - 38ms/epoch - 9ms/step
Epoch 10/80
4/4 - 0s - loss: 0.0287 - val_loss: 0.0186 - 37ms/epoch - 9ms/step
Epoch 11/80
4/4 - 0s - loss: 0.0249 - val_loss: 0.0198 - 35ms/epoch - 9ms/step
Epoch 12/80
4/4 - 0s - loss: 0.0255 - val_loss: 0.0209 - 37ms/epoch - 9ms/step
Epoch 13/80
4/4 - 0s - loss: 0.0246 - val_loss: 0.0214 - 31m

In [16]:
models_df = pd.DataFrame.from_dict(models, orient='index', columns=['RMSE', 'MSE', 'MAE', 'R_sq', 'MAPE'])
models_df

,RMSE,MSE,MAE,R_sq,MAPE
40,4116.922385,1.694905e+07,3314.483088,0.189030,0.193665
70,4227.638616,1.787293e+07,3416.701455,0.144824,0.202435
100,4170.355515,1.739187e+07,3385.989067,0.167842,0.199793
